In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-211491
Azure region: southcentralus
Subscription id: 3d1a56d2-7c81-4118-9790-f85d1acf0c77
Resource group: aml-quickstarts-211491


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "My-Cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### MY CODE HERE ###
# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                            max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
### My CODE HERE ###
ps = RandomParameterSampling( {
        "--C": uniform(0.1, 5.0),           # regularization
        "--max_iter": choice(20, 50, 80)    # max number of iterations used
    }
)

# Specify a Policy
### My CODE HERE ###
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
### My CODE HERE ###
src = ScriptRunConfig(source_directory=".",
                      script='train.py',
                      compute_target=cpu_cluster,
                      environment=sklearn_env)


# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
### My CODE HERE ###
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4,
                                     )

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_e74abb6b-6dc5-4e57-a421-44c382884235
Web View: https://ml.azure.com/runs/HD_e74abb6b-6dc5-4e57-a421-44c382884235?wsid=/subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourcegroups/aml-quickstarts-211491/workspaces/quick-starts-ws-211491&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2022-10-15T12:51:21.230116][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2022-10-15T12:51:22.0736813Z][SCHEDULER][INFO]Scheduling job, id='HD_e74abb6b-6dc5-4e57-a421-44c382884235_0' 
[2022-10-15T12:51:22.3137724Z][SCHEDULER][INFO]Scheduling job, id='HD_e74abb6b-6dc5-4e57-a421-44c382884235_2' 
[2022-10-15T12:51:22.2473557Z][SCHEDULER][INFO]Scheduling job, id='HD_e74abb6b-6dc5-4e57-a421-44c382884235_1' 
[2022-10-15T12:51:22.331628][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2022-10-15T12:51:22.3921527Z][SCHEDULER][INFO]Scheduling job, id='HD_e74abb6b-6dc5-4e57-a421-44c3828842

{'runId': 'HD_e74abb6b-6dc5-4e57-a421-44c382884235',
 'target': 'My-Cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-10-15T12:51:20.52957Z',
 'endTimeUtc': '2022-10-15T13:13:21.701715Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '873829a9-1bce-463d-b633-c9a408bdec18',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1017-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.44.0',
  'space_size': 'infinite_space_size',
  'score': '0.9099295945617868',
  'best_child_run_id': 'HD_e74abb6b-6dc5-4e57-a421-44c382884235_6',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_e74abb6b-6dc5-4e57-a421-44c382884235_6'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues

In [9]:
import joblib
# Get your best run and save the model from that run.

### My CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n parameter values:',parameter_values)

print(best_run.get_file_names())
filename = 'best_hyper_drive_model'
best_model = best_run.register_model(model_name=filename, model_path="./")

Best Run Id:  HD_e74abb6b-6dc5-4e57-a421-44c382884235_6

 Accuracy: 0.9099295945617868

 parameter values: ['--C', '3.5732050875144887', '--max_iter', '80']
['logs/azureml/dataprep/0/backgroundProcess.log', 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/0/rslex.log.2022-10-15-13', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/metrics_capability/metrics-capability.log', 'system_logs/snapshot_capability/snapshot-capability.log', 'user_logs/std_log.txt']


In [10]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
tds_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### My CODE HERE ###
tds = TabularDatasetFactory.from_delimited_files(tds_path)

In [11]:
from train import clean_data
import pandas as pd

# Use the clean_data function to clean your data.
### My DATA OBJECT HERE ###
x, y = clean_data(tds)

# the output of the clean_data function seems to be aimed at the deprecated way of inputting training_data
# so I have to put it back together
df = pd.concat([x, y], axis=1)
datastore=ws.get_default_datastore()
training_data=TabularDatasetFactory.register_pandas_dataframe(df, datastore, 'training_data') 

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/1b2dfb78-6dad-403a-865d-3a51e838e292/
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'emp.var.rate' -> 'emp_var_rate'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'cons.price.idx' -> 'cons_price_idx'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'cons.conf.idx' -> 'cons_conf_idx'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'nr.employed' -> 'nr_employed'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'job_admin.' -> 'job_admin_'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'education_basic.4y'

In [21]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=training_data,
    label_column_name="y",
    n_cross_validations=4,
    compute_target=cpu_cluster)

In [22]:
# Submit your automl run

### My CODE HERE ###
automl_run = exp.submit(config=automl_config)
RunDetails(automl_run).show()

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_8504fe34-1ef4-47b7-98de-79abe226777a,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [27]:
automl_run.wait_for_completion(show_output=False)

{'runId': 'AutoML_8504fe34-1ef4-47b7-98de-79abe226777a',
 'target': 'My-Cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-10-15T13:45:14.247389Z',
 'endTimeUtc': '2022-10-15T14:26:29.104644Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '4',
  'target': 'My-Cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"3d1a56d2-7c81-4118-9790-f85d1acf0c77","resource_group":"aml-quickstarts-211491","workspace_name":"quick-starts-ws-211491","region":"southcentralus","compute_target":"My-Cluster","spa

In [28]:
best_run, best_model = automl_run.get_output()
best_run.register_model(model_name= 'best_model_automl.pkl', model_path= './outputs/')

2022-10-15:14:26:52,936 WARNING  [package_utilities.py:198] The version of the SDK does not match the version the model was trained on.
2022-10-15:14:26:52,937 WARNING  [package_utilities.py:199] The consistency in the result may not be guaranteed.
2022-10-15:14:26:52,938 WARNING  [package_utilities.py:204] Package:azureml-automl-core, training version:1.46.1, current version:1.44.0
Package:azureml-automl-runtime, training version:1.46.1, current version:1.44.0
Package:azureml-core, training version:1.46.0, current version:1.44.0
Package:azureml-dataprep, training version:4.5.7, current version:4.2.2
Package:azureml-dataprep-rslex, training version:2.11.4, current version:2.8.1
Package:azureml-dataset-runtime, training version:1.46.0, current version:1.44.0
Package:azureml-defaults, training version:1.46.0, current version:1.44.0
Package:azureml-interpret, training version:1.46.0, current version:1.44.0
Package:azureml-mlflow, training version:1.46.0, current version:1.44.0
Package:azu

Model(workspace=Workspace.create(name='quick-starts-ws-211491', subscription_id='3d1a56d2-7c81-4118-9790-f85d1acf0c77', resource_group='aml-quickstarts-211491'), name=best_model_automl.pkl, id=best_model_automl.pkl:2, version=2, tags={}, properties={})

In [32]:
best_run_metrics = best_run.get_metrics()


print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])

Best Run Id:  AutoML_8504fe34-1ef4-47b7-98de-79abe226777a_30

 Accuracy: 0.917480930245554


In [33]:
print('\n all metrics of best run:', best_run_metrics)


 all metrics of best run: {'precision_score_micro': 0.917480930245554, 'AUC_micro': 0.9804870561324069, 'average_precision_score_macro': 0.8247611478092767, 'f1_score_micro': 0.917480930245554, 'recall_score_weighted': 0.917480930245554, 'recall_score_micro': 0.917480930245554, 'accuracy': 0.917480930245554, 'AUC_macro': 0.9474152687676999, 'precision_score_weighted': 0.9146740236832603, 'average_precision_score_weighted': 0.9553564503758015, 'weighted_accuracy': 0.953032924705651, 'f1_score_macro': 0.7847150635653795, 'log_loss': 0.2549733118473436, 'precision_score_macro': 0.7965182568699452, 'f1_score_weighted': 0.9158922421094131, 'AUC_weighted': 0.9474152687677, 'matthews_correlation': 0.5703636538019725, 'recall_score_macro': 0.7743543281188681, 'norm_macro_recall': 0.5487086562377362, 'balanced_accuracy': 0.7743543281188681, 'average_precision_score_micro': 0.9812164914980489, 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_8504fe34-1ef4-47b7-98de-79abe226777a_30/

In [34]:
# Proof of cluster cleanup
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing cluster, deleting it.")
    cpu_cluster.delete()
except ComputeTargetException:
    print("Cluster not found.")

Found existing cluster, deleting it.
